# JunctionTree

A `JunctionTree` is an intermediate data structure used in GTSAM's multifrontal variable elimination. It is a `ClusterTree` where each node (cluster) corresponds to a clique in the chordal graph formed during elimination.

Key differences from related structures:
*   **vs. EliminationTree:** Junction tree nodes can represent the elimination of multiple variables simultaneously (a 'frontal' set), whereas elimination tree nodes typically represent single variable eliminations.
*   **vs. BayesTree:** A JunctionTree node contains the original factors associated with the variables being eliminated in that clique. A BayesTree node contains the *result* of eliminating those factors (i.e., a conditional density $P(	ext{Frontals} | 	ext{Separator})$).

Like `EliminationTree`, direct manipulation of `JunctionTree` objects in Python is uncommon. It's primarily an internal structure used by `eliminateMultifrontal` when producing a `BayesTree`.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/JunctionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [ ]:
import gtsam
import numpy as np

# JunctionTree is templated, need concrete types
from gtsam import GaussianFactorGraph, Ordering, GaussianJunctionTree, GaussianBayesTree
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Creating a JunctionTree

A `JunctionTree` is typically constructed from an `EliminationTree` as part of the multifrontal elimination process. The direct constructor might not be exposed in Python, as it's usually created internally.

In [ ]:
# Create a graph (same as BayesTree example)
graph = GaussianFactorGraph()
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
graph.add(X(0), -np.eye(1), np.zeros(1), model)
graph.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(X(1), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(0), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)

ordering = Ordering.Colamd(graph)

# Perform multifrontal elimination, which uses a JunctionTree internally
bayes_tree, remaining_graph = graph.eliminatePartialMultifrontal(ordering)

# The resulting BayesTree reflects the structure of the intermediate JunctionTree
print("Resulting BayesTree (structure mirrors JunctionTree):")
bayes_tree.print()

# Accessing the JunctionTree directly isn't typical in Python workflows.
# Its structure is implicitly captured by the BayesTree cliques.

Resulting BayesTree (structure mirrors JunctionTree):
: cliques: 3, variables: 5
Root(s):
Conditional density P(l1, x1 | l2, x2) = P(l1 | x1) P(x1 | l2, x2) 
  size: 2
  Conditional P(l1 | x1): GaussianConditional( P(l1 | x1) = dl1 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.866025 ])

  Conditional P(x1 | l2, x2): GaussianConditional( P(x1 | l2, x2) = dx1 - R1*dl2 - R2*dx2 - d), R1 = [ 0.333333 ], R2 = [ 0.333333 ], d = [ 0 ], sigmas = [ 0.745356 ])


  Children:
  Conditional density P(l2, x2 | x0) = P(l2 | x2) P(x2 | x0) 
    size: 2
    Conditional P(l2 | x2): GaussianConditional( P(l2 | x2) = dl2 - R*dx2 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.866025 ])

    Conditional P(x2 | x0): GaussianConditional( P(x2 | x0) = dx2 - R*dx0 - d), R = [ 0.2 ], d = [ 0 ], sigmas = [ 0.774597 ])


    Children:
    Conditional density P(x0) = P(x0) 
      size: 1
      Conditional P(x0): GaussianConditional( P(x0) = dx0 - d), d = [ 0 ], sigmas = [ 0.894427 ])



